In [19]:
#import stokespy
#from stokespy import get_HMI_data, get_SP_data, StokesParamCube, StokesParamMap, StokesProfile
import datetime
import astropy
import astropy.units as u
from astropy.coordinates import SkyCoord, SpectralCoord
from astropy.wcs.wcsapi import SlicedLowLevelWCS, HighLevelWCSWrapper

import sunpy.map
from sunpy.net import Fido, attrs

from astropy.io import fits
import aiapy
from aiapy.calibrate import register, update_pointing

import numpy as np
import os
import natsort
import matplotlib.pyplot as plt
import matplotlib
#import ndcube

matplotlib.use('TkAgg')

# Setup the autoreload function.
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def parse_folder(dir_path=None, inst=None, wave=None, ext=None, 
                 series=None, repo='JSOC', show=True):
    """
    Search all the filenames in a folder containing SDO data and use the keywords
    to select a desired subset. For example setting inst="hmi" will obtain all the hmi filenames.
    dir_path: Path to the directory containing the data.
    inst: SDO instrument
    wave: wavelength (primarily for AIA data)
    ext: Select the file extension
    series: String characterizing the data series (e.g. hmi.S_720s, aia.lev1_euv_12s)
    repo: Choose the data repository. Each repository stores filenames with different syntaxes.
    show: Flag that enumerates the files found.
    """
    
    if dir_path is None:
        dir_path = os.getcwd()
    
    # Read and sort all the filenames in the folder.
    all_fnames = natsort.natsorted(os.listdir(dir_path)) 
    
    # Select a subset of files
    use_fnames = []
    
    for i, file_ in enumerate(all_fnames):
        if repo == 'JSOC':
            sfile_ = file_.lower().split(".")
        elif repo == 'VSO':
            sfile_ = file_.lower().split("_")
        
        if sfile_[0] == inst.lower() and sfile_[1] == series.lower(): 
            use_fnames.append(dir_path + file_)
        
        '''
        if ext is None and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        '''
    '''
    for i, file_ in enumerate(all_fnames):
        # Select by extension.
        if ext is not None and file_.endswith(ext):        
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
        else:
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
    '''
    
    if show:
        for i, file_ in enumerate(use_fnames):
            print(i, file_)
    
    return use_fnames

In [6]:
# Select a date and time to search for observations.
#user_date = astropy.time.Time(datetime.datetime(2016, 7, 28, 23, 57, 0), scale='tai')  # Original
#user_date = astropy.time.Time(datetime.datetime(2017, 8, 22, 17, 11, 4), scale='tai')  # AR near limb
user_date = astropy.time.Time(datetime.datetime(2017, 9, 5, 3, 4, 4), scale='tai')  # AR near disc center

# Find the nearest set of Stokes and corresponding inversion results.
#all_fnames_stokes, all_fnames_magvec = get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=False)
#lv1_data, lv1_wcs, lv2_data, lv2_wcs = \
#    get_HMI_data(user_date, user_notify='gdima@hawaii.edu', download=False, show_files=False)

time0 = astropy.time.Time(user_date.gps - 1., format='gps', scale='tai')
time1 = astropy.time.Time(user_date.gps + 1., format='gps', scale='tai')

user_dir = os.getcwd() + '/Data/SDO/'

all_fnames_stokes = parse_folder(dir_path=user_dir, inst='hmi', \
                                 series='S_720s', ext='fits', show=False)

if len(all_fnames_stokes) > 1:
    tstamps = [i.split('.')[2] for i in all_fnames_stokes]
    tstamps = [sunpy.time.parse_time('_'.join(i.split('_')[0:2])) for i in tstamps]
    tstamps_diff = [np.abs(i.gps - user_date.gps) for i in tstamps]

# Search for the closest timestamp
tstamps_diff = np.asarray(tstamps_diff)
tstamps_ix, = np.where(tstamps_diff == tstamps_diff.min())

all_fnames_stokes = np.asarray(all_fnames_stokes)[tstamps_ix]

In [7]:
## Create data array
## Use sunpy.map.Map to read HMI files since it provides the correct observer frame of reference.

level1_data = []
for i, fname in enumerate(all_fnames_stokes):
    level1_data.append(sunpy.map.Map(fname).data)

level1_data = np.asarray(level1_data)
level1_data = level1_data.reshape(4,6,level1_data.shape[1], level1_data.shape[2])

print(f'Created data cube with dimensions: {level1_data.shape}')

Created data cube with dimensions: (4, 6, 4096, 4096)


In [8]:
## Create the WCS object
# Expand the coordinate axis to include wavelength and stokes dimensions.

l0 = 6173.345 * 1.e-10  # m Central wavelength for FeI line
dl = 0.0688   * 1.e-10  # m 

# Generate WCS for data cube using same WCS celestial information from AIA map.
wcs_header = sunpy.map.Map(all_fnames_stokes[0]).wcs.to_header()

wcs_header["WCSAXES"] = 4

# Add wavelength axis.
wcs_header["CRPIX3"] = 3.5
wcs_header["CDELT3"] = dl
wcs_header["CUNIT3"] = 'm'
wcs_header["CTYPE3"] = "WAVE"
wcs_header["CRVAL3"] = l0

# Add Stokes axis.
wcs_header["CRPIX4"] = 0
wcs_header["CDELT4"] = 1
wcs_header["CUNIT4"] = ''
wcs_header["CTYPE4"] = "STOKES"
wcs_header["CRVAL4"] = 0

level1_wcs = astropy.wcs.WCS(wcs_header)

In [9]:
level1_wcs

WCS Keywords

Number of WCS axes: 4
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'WAVE'  'STOKES'  
CRVAL : 0.0  0.0  6.173345e-07  0.0  
CRPIX : 2040.244140625  2050.626953125  3.5  0.0  
PC1_1 PC1_2 PC1_3 PC1_4  : -0.99999997235056  0.00023515712208659  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : -0.00023515712208659  -0.99999997235056  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : 0.00014009532001283  0.00014009532001283  6.88e-12  1.0  
NAXIS : 0  0

In [10]:
# Generate WCS for data cube using same WCS celestial information from AIA map.
#wcs_header = sunpy.map.Map(all_fnames_stokes[0]).wcs.to_header()
hmi_image = sunpy.map.Map(all_fnames_stokes[0])

In [11]:
hmi_image.wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  
CRVAL : 0.0  0.0  
CRPIX : 2040.244140625  2050.626953125  
PC1_1 PC1_2  : -0.99999997235056  0.00023515712208659  
PC2_1 PC2_2  : -0.00023515712208659  -0.99999997235056  
CDELT : 0.00014009532001283  0.00014009532001283  
NAXIS : 0  0

In [12]:
# Rotate, scale (0.5 -> 0.6 arcsec) and center the map
hmi_image_reg = register(hmi_image)

In [13]:
hmi_image_reg.wcs

WCS Keywords

Number of WCS axes: 2
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  
CRVAL : 0.0  0.0  
CRPIX : 2048.5  2048.5  
PC1_1 PC1_2  : 1.0  7.4040177438477e-22  
PC2_1 PC2_2  : 7.4040177438477e-22  1.0  
CDELT : 0.00016666666666667  0.00016666666666667  
NAXIS : 0  0

In [21]:
hmi_image.wcs.world_axis_units

['deg', 'deg']

In [38]:
hmi_image.scale[0]/(0.6*u.arcsec)

<Quantity 0.84057192 1 / pix>

In [39]:
new_2d_slice = [0]*(4-2)

In [42]:
new_2d_slice

[0, 0, slice(None, None, None), slice(None, None, None)]

In [41]:
new_2d_slice.extend([slice(None), slice(None)])

In [16]:
hmi_image.wcs.to_header()

WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =       2040.244140625 / Pixel coordinate of reference point            
CRPIX2  =       2050.626953125 / Pixel coordinate of reference point            
PC1_1   =    -0.99999997235056 / Coordinate transformation matrix element       
PC1_2   =  0.00023515712208659 / Coordinate transformation matrix element       
PC2_1   = -0.00023515712208659 / Coordinate transformation matrix element       
PC2_2   =    -0.99999997235056 / Coordinate transformation matrix element       
CDELT1  =  0.00014009532001283 / [deg] Coordinate increment at reference point  
CDELT2  =  0.00014009532001283 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'HPLN-TAN'           / Coordinate type codegnomonic projection        
CTYPE2  = 'HPLT-TAN'        

In [17]:
hmi_image?

Type:        HMIMap
String form:
SunPy Map
           ---------
           Observatory:		 SDO
           Instrument:		 HMI COMBINED
           Detector:		 HMI
           Measurement:		  <...>  [nan, nan, nan, ..., nan, nan, nan],
           [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)
File:        ~/anaconda3/envs/stokespy/lib/python3.9/site-packages/sunpy/map/sources/sdo.py
Docstring:  
HMI Image Map.

HMI consists of a refracting telescope, a polarization selector,
an image stabilization system, a narrow band tunable filter
and two 4096 pixel CCD cameras. It observes the full solar disk in the Fe I
absorption line at 6173 Angstrom with a resolution of 1 arc-second.
HMI takes images in a sequence of tuning and polarizations at a 4-second
cadence for each camera. One camera is dedicated to a 45 s Doppler and
line-of-sight field sequence while the other to a 90 s vector field
sequence.

References
----------
* `SDO Mission Page <https://sdo.gsfc.nasa.gov/>`_
* `Instrument Page <

In [20]:
hmi_image_f = fits.open(all_fnames_stokes[0])

In [29]:
hmi_image_f[0].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / number of bits per data pixel                  
NAXIS   =                    0 / number of data axes                            
EXTEND  =                    T / FITS dataset may contain extensions            
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H 

In [30]:
hmi_image_f[1].header

SIMPLE  =                    T / file does conform to FITS standard             
BITPIX  =                   16 / data type of original image                    
NAXIS   =                    2 / dimension of original image                    
NAXIS1  =                 4096 / length of original image axis                  
NAXIS2  =                 4096 / length of original image axis                  
PCOUNT  =                    0 / size of special data area                      
GCOUNT  =                    1 / one data group (required keyword)              
XTENSION= 'BINTABLE'           / binary table extension                         
BLANK   =               -32768                                                  
BZERO   =               97304.                                                  
BSCALE  =                   3.                                                  
CHECKSUM= '3gSnAZQn4dQnAZQn'   / HDU checksum updated 2022-01-19T06:24:36       
DATE    = '2021-03-20T14:53:

In [45]:
hmi_image.rotation_matrix

array([[-9.99999972e-01,  2.35157122e-04],
       [-2.35157122e-04, -9.99999972e-01]])